In [7]:
import numpy as np
import pandas as pd
import numba as nb

In [2]:
N, n = 10000, 10000
pct1, pct99 = 100, 9900
arr = np.random.randn(n, N)

In [16]:
def risk_measure_np(arr):
    arr_sort = np.sort(arr)
    VaR = arr_sort[:, [pct1, pct99]]
    left_tail, right_tail = arr_sort[:, :pct1], arr_sort[:, pct99:]
    ES1, ES99 = np.mean(left_tail, axis=1), np.mean(right_tail, axis=1)
    ES = np.stack((ES1, ES99), axis=1)
    return VaR, ES

In [17]:
%timeit VaR, ES = risk_measure_np(arr)
# display(VaR, ES)

5.56 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
@nb.jit
def risk_measure_np(arr):
    arr_sort = np.sort(arr, axis=1)
    return arr_sort

In [30]:
arr

array([[-0.11573952,  0.34097428,  1.12739016, ..., -0.38640829,
        -0.55641647,  0.20277826],
       [ 0.93823465, -1.18568809,  0.22206313, ..., -0.55630954,
         0.97654478, -1.02695767],
       [ 0.2127753 ,  0.61543232, -2.64875197, ..., -1.40566966,
         0.28255733, -0.1390713 ],
       ...,
       [-0.61799798,  2.48748143,  0.39549937, ..., -0.74864695,
        -0.4805876 , -0.32056304],
       [ 0.88140629,  0.33988739,  0.67369271, ..., -0.12394958,
        -1.65849982,  0.76585789],
       [ 1.67815214,  1.37611418,  0.94984644, ...,  1.37083817,
        -0.38327814,  0.16744417]])

In [29]:
%timeit risk_measure_np(arr)
# display(VaR, ES)

/var/folders/z_/3qxnq2j91195ff29hsnwv7w80000gn/T/ipykernel_80946/1903543859.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "risk_measure_np" failed type inference due to: No implementation of function Function(<function sort at 0x7f8160fbd1f0>) found for signature:
 
 >>> sort(array(float64, 2d, C), axis=Literal[int](1))
 
There are 4 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 2d, C), axis=int64)':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 2d, C), int64)
        
       There are 2 candidate implementations:
         -

5.47 s ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
def var_func(x):
    x.sort_values(inplace=True)
    return [x.iloc[pct1], x.iloc[pct99]]
def es_func(x):
    x.sort_values(inplace=True)
    return [x.iloc[:pct1].mean(), x.iloc[pct99:].mean()]

def risk_measure_pd(arr):
    df = pd.DataFrame(arr)
    VaR = df.apply(var_func, axis=1).to_list()
    ES = df.apply(es_func, axis=1).to_list()
    return VaR, ES

In [6]:
%timeit VaR, ES = risk_measure_pd(arr)
display(VaR, ES)

15.1 s ± 308 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


NameError: name 'VaR' is not defined